In [1]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 138638, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 138638 (delta 94), reused 87 (delta 32), pack-reused 138439
Receiving objects: 100% (138638/138638), 136.99 MiB | 22.82 MiB/s, done.
Resolving deltas: 100% (103915/103915), done.


In [1]:
!pip install -e /content/transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.29.0.dev0-0.editable-py3-none-any.whl size=36076 sha256=6689ea90db71b7002d856555f5d43ada157f7ee56ed1e0e0917c841d12477d8f
  Stored in directory: /tmp/pip-ephem-wheel-cache-5f9cl3fx/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.0.dev0
    Uninstalling transformers-4.29.0.dev0:
      Successfully uninstalled transformers-4.29.0.dev0


In [2]:
!pip install datasets seqeval evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd 
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import RobertaForSequenceClassification, TrainingArguments, Trainer
from datasets import load_metric, load_dataset
from tqdm.notebook import tqdm
import torch


In [4]:
dataset = load_dataset('imdb')

  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [6]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.encoder.layer.9.attention.ou

In [35]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset, DatasetDict
train_data, small_data = train_test_split(dataset['train'], train_size=1000, random_state=42)
from datasets import Dataset
val_data, _ = train_test_split(dataset['test'], train_size=1000, random_state=42)
# Convert the split data back to DatasetDict format
small_dataset = DatasetDict({"train": train_data})
val_dataset = DatasetDict({"val": val_data})
df = pd.DataFrame(small_dataset['train'], columns=['text', 'label'])
df_ = pd.DataFrame(val_dataset['val'], columns=['text', 'label'])
train_small_dataset = Dataset.from_pandas(df)
val_small_dataset = Dataset.from_pandas(df_)


In [36]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = train_small_dataset.map(tokenize_function, batched=True)
tokenized_val = val_small_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [10]:
import re
for name,  param in model.named_parameters():
  if not re.search(r'Adapter|classifier', name):
    param.requires_grad = False
  else:
    param.requires_grad = True

In [ ]:
updating_parameters = []
frozen_parameters = []
for name, param in model.named_parameters():
    if param.requires_grad:
        updating_parameters.append(name)
    else:
        frozen_parameters.append(name)
for param_name in updating_parameters:
    print(param_name)

for param_name in frozen_parameters:
    print(param_name)


In [33]:
import numpy as np
import evaluate

acc = evaluate.load('accuracy')
f1 = evaluate.load('f1')
recall = evaluate.load('recall')
precision = evaluate.load('precision')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    #print(len(predictions), len(labels))
    acc_r = acc.compute(predictions=predictions, references=labels)
    f_r = f1.compute(predictions=predictions, references=labels)
    r_r = recall.compute(predictions=predictions, references=labels)
    p_r = precision.compute(predictions=predictions, references=labels)
    #print(results)
    return {"precision": p_r["precision"], "recall": r_r["recall"], "f1": f_r["f1"], "accuracy": acc_r["accuracy"]}

In [37]:
training_args = TrainingArguments(
    output_dir="D_Adapter_model",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/content/transformers/src/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.615745,0.854545,0.846000,0.850251,0.851000
2,No log,0.619445,0.845703,0.866000,0.855731,0.854000
3,No log,0.524719,0.806798,0.902000,0.851747,0.843000
4,No log,0.545789,0.873418,0.828000,0.850103,0.854000
5,No log,0.535989,0.861507,0.846000,0.853683,0.855000


TrainOutput(global_step=315, training_loss=0.12570064484126983, metrics={'train_runtime': 595.2964, 'train_samples_per_second': 8.399, 'train_steps_per_second': 0.529, 'total_flos': 1518730243130112.0, 'train_loss': 0.12570064484126983, 'epoch': 5.0})

In [51]:
# Generate patch files for the last three commits
!git format-patch -1 HEAD

fatal: not a git repository (or any of the parent directories): .git


In [50]:
!git checkout -b transformers
!git add .
!git commit -m "My changes to the model"

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
